# Explore Fastdb class

In [ ]:
from IPython.display import display, HTML 

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

## Why

So far I have created two demos, a simple one on `fastdebug.utils.whatinside` and a real world one on `fastcore.meta.FixSigMeta`. They should be enough as demos to show the usefulness of fastdebug library. 

However, I want to be thorough and really prove to myself that it can help me to conquer unknowns and clear doubts with ease and joy when exploring source code. 

Fastdb class and the funcs it uses contains all the tricks I learnt and difficulties I conquered which I don't always remember how and why. In fact I don't want to remember them. I want to just run all cells here and skim through the notebook and everything makes sense to me. 

Can fastdebug help me achieve it? Let's give a try!

## How to do it?

I need a few examples first. Maybe I could just use the simple demo as example for `Fastdb`.

## `whatinside` and `gw['whatinside']` are the same

In [ ]:
from fastdebug.utils import *
import inspect

In [ ]:
gw = {}
gw.update(whatinside.__globals__)
len(gw)

20

In [ ]:
whatinside.__code__.__repr__()

'<code object whatinside at 0x103b9f0e0, file "/Users/Natsume/Documents/fastdebug/fastdebug/utils.py", line 14>'

In [ ]:
gw['whatinside'].__code__

<code object whatinside at 0x103b9f0e0, file "/Users/Natsume/Documents/fastdebug/fastdebug/utils.py", line 14>

In [ ]:
inspect.getsource(gw['whatinside'])

'def whatinside(mo, # module, e.g., `import fastcore.all as fa`, use `fa` here\n               dun:bool=False, # print all items in __all__\n               func:bool=False, # print all user defined functions\n               clas:bool=False, # print all class objects\n               bltin:bool=False, # print all builtin funcs or methods\n               lib:bool=False, # print all the modules of the library it belongs to\n               cal:bool=False # print all callables\n             ): \n    \'Check what inside a module: `__all__`, functions, classes, builtins, and callables\'\n    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0\n    funcs = inspect.getmembers(mo, inspect.isfunction)\n    classes = inspect.getmembers(mo, inspect.isclass)\n    builtins = inspect.getmembers(mo, inspect.isbuiltin)\n    callables = inspect.getmembers(mo, callable)\n    pkgpath = os.path.dirname(mo.__file__)\n    print(f"{mo.__name__} has: \\n{dun_all} items in its __all__, and \\n{len(funcs)} 

In [ ]:
gw['whatinside'] == whatinside

True

## Dot it in a more natural and ordered way

In [ ]:
from fastdebug.core import *

In [ ]:
import inspect

### Prepare env for Fastdb.dbprint

In [ ]:
g = {}
g.update(Fastdb.dbprint.__globals__)
len(g)

157

In [ ]:
'Fastdb.dbprint' in g

False

In [ ]:
'Fastdb' in g

True

In [ ]:
'dbprint' in g 

True

In [ ]:
type(g['dbprint']) # has nothing, this is probably come from the notebook 00_core

NoneType

#### Fastdb.dbprint and `g['Fastdb'].dbprint` are the same

In [ ]:
g['Fastdb'].dbprint

<function fastdebug.core.Fastdb.dbprint(self: fastdebug.core.Fastdb, dbcode: int, cmt: str, *codes, expand: int = 2, showdbsrc: bool = False)>

In [ ]:
inspect.getdoc(g['Fastdb'].dbprint)

'Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating'

In [ ]:
Fastdb.dbprint == g['Fastdb'].dbprint

True

In [ ]:
inspect.getsource(Fastdb.dbprint)

'@patch\ndef dbprint(self:Fastdb, \n            dbcode:int, # idx of a srcline under investigation, can only be int\n            cmt:str, # comment added to the srcline\n            *codes, # a list of expressions (str) you write to be evaluated above the srcline\n            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated\n            showdbsrc:bool=False): # display dbsrc\n    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"\n\n    src = self.orisrc\n    if type(dbcode) == int: self.cmts.update({dbcode: cmt})\n\n    printsrc(src, dbcode, cmt, expand)\n    print(\'{:-<60}\'.format(colorize("print selected srcline with expands above", color="y")))\n    \n    dbsrc = ""\n    indent = 4\n    onedbprint = False\n\n    lst = inspect.getsource(src).split(\'\\n\')\n    if not bool(lst[-1]): lst = lst[:-1]\n\n    newlst = []\n    for i in codes: # no matter whether there is "" or "  "

### Create Fastdb object for `Fastdb.dbprint`

In [ ]:
fdb = Fastdb(Fastdb.dbprint, g)

In [ ]:
fdb.print(10, 1)

@patch====================================================================================(0)       
def dbprint(self:Fastdb, =================================================================(1)       
            dbcode:int, # idx of a srcline under investigation, can only be int===========(2)       
            cmt:str, # comment added to the srcline=======================================(3)       
            *codes, # a list of expressions (str) you write to be evaluated above the srcline                                                           (4)
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated                                                            (5)
            showdbsrc:bool=False): # display dbsrc========================================(6)       
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                           

Important! After running the following line, `Fastdb.dbprint` is updated by exec and source code can't be seen anymore.

In [ ]:
dbsrc = fdb.dbprint(9, "keep original src safe", "self.orisrc", showdbsrc=True)

    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                                                                                                                                                        (8)
    src = self.orisrc===================================================================================================================================(9)
                                                                                                                                       keep original src safe
    if type(dbcode) == int: self.cmts.update({dbcode: cmt})                                                                                             (10)
                                                                                                                                                        (11)
print selected srcline with expands above----------
showdbsr

Important!

As updating `Fastdb.dbprint` with exec will send the updated src to the class `Fastdb` in `g`, the newly created `dbprint` in locals() is actually NoneType.

Luckily, we still got `fdb.orisrc` which keep the original `Fastdb.dbprint` for us.

So, we can run `Fastdb.dbprint = fdb.orisrc` to get `Fastdb.dbprint` back to normal

### Make an example with new `Fastdb.dbprint`

In [ ]:
from fastdebug.utils import *

In [ ]:
import fastdebug.core as core

In [ ]:
gw = {}
gw.update(whatinside.__globals__)
len(gw)

20

In [ ]:
fdbw = Fastdb(whatinside, gw) # we are using the updated Fastdb whose dbprint has dbprintinsert() ready

In [ ]:
fdbw.print(10,1) 

def whatinside(mo, # module, e.g., `import fastcore.all as fa`, use `fa` here=============(0)       
               dun:bool=False, # print all items in __all__===============================(1)       
               func:bool=False, # print all user defined functions========================(2)       
               clas:bool=False, # print all class objects=================================(3)       
               bltin:bool=False, # print all builtin funcs or methods=====================(4)       
               lib:bool=False, # print all the modules of the library it belongs to=======(5)       
               cal:bool=False # print all callables=======================================(6)       
             ): ==========================================================================(7)       
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'==(8)       
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0============================(

In [ ]:
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)") # the updated dbprint will run here.

whatinside = new
whatinside(core)



                                                                                            self.orisrc => self.orisrc : <function whatinside at 0x103c32940>
             ):                                                                                                                                         (7)
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'                                                                (8)
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0==========================================================================================(9)
                                                                                                                                count num of funcs in __all__
    funcs = inspect.getmembers(mo, inspect.isfunction)                                                                                                  (10)
    classes = inspect.getmembers(mo, inspect.isclass)    

#### Turn Fastdb.dbprint and whatinside back to normal

In [ ]:
Fastdb.dbprint = fdb.orisrc
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration

### explore the second srcline of dbprint

In [ ]:
fdb.print(20, 1) # we are using fdb.orisrc to print, so no dbprintinsert() will be seen.

@patch====================================================================================(0)       
def dbprint(self:Fastdb, =================================================================(1)       
            dbcode:int, # idx of a srcline under investigation, can only be int===========(2)       
            cmt:str, # comment added to the srcline=======================================(3)       
            *codes, # a list of expressions (str) you write to be evaluated above the srcline                                                           (4)
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated                                                            (5)
            showdbsrc:bool=False): # display dbsrc========================================(6)       
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                           

In [ ]:
# this line will udpate or change Fastdb.dbprint again
fdb.dbprint(10, "collect and organize cmt by idx", "self.cmts", "dbcode", "cmt", showdbsrc=True)

                                                                                                                                                        (8)
    src = self.orisrc                                                                                                                                   (9)
    if type(dbcode) == int: self.cmts.update({dbcode: cmt})=============================================================================================(10)
                                                                                                                              collect and organize cmt by idx
                                                                                                                                                        (11)
    printsrc(src, dbcode, cmt, expand)                                                                                                                  (12)
print selected srcline with expands above----------
showdbs

In [ ]:
# Now, use the newly created or updated Fastdb to create an object for debugging whatinside
fdbw = Fastdb(whatinside, gw)
fdbw.print(10,1)

def whatinside(mo, # module, e.g., `import fastcore.all as fa`, use `fa` here=============(0)       
               dun:bool=False, # print all items in __all__===============================(1)       
               func:bool=False, # print all user defined functions========================(2)       
               clas:bool=False, # print all class objects=================================(3)       
               bltin:bool=False, # print all builtin funcs or methods=====================(4)       
               lib:bool=False, # print all the modules of the library it belongs to=======(5)       
               cal:bool=False # print all callables=======================================(6)       
             ): ==========================================================================(7)       
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'==(8)       
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0============================(

In [ ]:
# we are still debugging the same line in whatinside
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
whatinside = new
whatinside(core)



                                                                                                                                  self.cmts => self.cmts : {}


                                                                                                                                         dbcode => dbcode : 9


                                                                                                                   cmt => cmt : count num of funcs in __all__
             ):                                                                                                                                         (7)
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'                                                                (8)
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0==========================================================================================(9)
                                                    

#### Turn Fastdb.dbprint and whatinside back to normal

In [ ]:
Fastdb.dbprint = fdb.orisrc
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration

### Explore the third line

In [ ]:
fdb.print(20, 1)

@patch====================================================================================(0)       
def dbprint(self:Fastdb, =================================================================(1)       
            dbcode:int, # idx of a srcline under investigation, can only be int===========(2)       
            cmt:str, # comment added to the srcline=======================================(3)       
            *codes, # a list of expressions (str) you write to be evaluated above the srcline                                                           (4)
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated                                                            (5)
            showdbsrc:bool=False): # display dbsrc========================================(6)       
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                           

In [ ]:
# update a new Fastdb.dbprint
fdb.dbprint(12, "print srcdoe with comment, idx and specified expands", "inspect.getdoc(printsrc)")
# creating a db object on whatinside with new Fastdb.dbprint
fdbw = Fastdb(whatinside, gw)
# fdbw.print(10, 1)
# we are still debuggin the same line in whatinside
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
whatinside = new
whatinside(core)
# Turn Fastdb.dbprint and whatinside back to normal
Fastdb.dbprint = fdb.orisrc
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration

    if type(dbcode) == int: self.cmts.update({dbcode: cmt})                                                                                             (10)
                                                                                                                                                        (11)
    printsrc(src, dbcode, cmt, expand)==================================================================================================================(12)
                                                                                                         print srcdoe with comment, idx and specified expands
    print('{:-<60}'.format(colorize("print selected srcline with expands above", color="y")))                                                           (13)
                                                                                                                                                        (14)
print selected srcline with expands above----------


   

### Explore the fourth line

In [ ]:
fdb.print(20, 1)

@patch====================================================================================(0)       
def dbprint(self:Fastdb, =================================================================(1)       
            dbcode:int, # idx of a srcline under investigation, can only be int===========(2)       
            cmt:str, # comment added to the srcline=======================================(3)       
            *codes, # a list of expressions (str) you write to be evaluated above the srcline                                                           (4)
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated                                                            (5)
            showdbsrc:bool=False): # display dbsrc========================================(6)       
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                           

In [ ]:
# update a new Fastdb.dbprint
fdb.dbprint(16, "make sure only one srcline is selected at a time")
# creating a db object on whatinside with new Fastdb.dbprint
fdbw = Fastdb(whatinside, gw)
# fdbw.print(10, 1)
# we are still debuggin the same line in whatinside
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
whatinside = new
whatinside(core)
# Turn Fastdb.dbprint and whatinside back to normal
Fastdb.dbprint = fdb.orisrc
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration

                                                                                                                                                        (14)
    dbsrc = ""                                                                                                                                          (15)
    indent = 4==========================================================================================================================================(16)
                                                                                                             make sure only one srcline is selected at a time
    onedbprint = False                                                                                                                                  (17)
                                                                                                                                                        (18)
print selected srcline with expands above----------
     

### Explore the fifth line

In [ ]:
fdb.print(20, 1)

@patch====================================================================================(0)       
def dbprint(self:Fastdb, =================================================================(1)       
            dbcode:int, # idx of a srcline under investigation, can only be int===========(2)       
            cmt:str, # comment added to the srcline=======================================(3)       
            *codes, # a list of expressions (str) you write to be evaluated above the srcline                                                           (4)
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated                                                            (5)
            showdbsrc:bool=False): # display dbsrc========================================(6)       
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                           

In [ ]:
# update a new Fastdb.dbprint
fdb.dbprint(19, "remove the last line if it is empty")
# creating a db object on whatinside with new Fastdb.dbprint
fdbw = Fastdb(whatinside, gw)
# fdbw.print(10, 1)
# we are still debuggin the same line in whatinside
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
whatinside = new
whatinside(core)
# Turn Fastdb.dbprint and whatinside back to normal
Fastdb.dbprint = fdb.orisrc
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration

    onedbprint = False                                                                                                                                  (17)
                                                                                                                                                        (18)
    lst = inspect.getsource(src).split('\n')============================================================================================================(19)
                                                                                                                          remove the last line if it is empty
    if not bool(lst[-1]): lst = lst[:-1]                                                                                                                (20)
                                                                                                                                                        (21)
print selected srcline with expands above----------
     

### Explore the 6th line

In [ ]:
fdb.print(15, 2)

    dbsrc = ""============================================================================(15)      
    indent = 4============================================================================(16) # make sure only one srcline is selected at a time
    onedbprint = False====================================================================(17)      
                                                                                                                                                        (18)
    lst = inspect.getsource(src).split('\n')==============================================(19) # remove the last line if it is empty
    if not bool(lst[-1]): lst = lst[:-1]==================================================(20)      
                                                                                                                                                        (21)
    newlst = []===========================================================================(22)      
   

In [ ]:
# update a new Fastdb.dbprint
fdb.dbprint(24, "users can put '' and '  ' as codes, but will be ignored.")
# creating a db object on whatinside with new Fastdb.dbprint
fdbw = Fastdb(whatinside, gw)
# fdbw.print(10, 1)
# we are still debuggin the same line in whatinside
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
whatinside = new
whatinside(core)
# Turn Fastdb.dbprint and whatinside back to normal
Fastdb.dbprint = fdb.orisrc
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration

    newlst = []                                                                                                                                         (22)
    for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes                                                      (23)
        if bool(i.strip()): newlst.append(i)============================================================================================================(24)
                                                                                                     users can put '' and '  ' as codes, but will be ignored.
    codes = newlst                                                                                                                                      (25)
                                                                                                                                                        (26)
print selected srcline with expands above----------
     

### Explore the 7th line

In [ ]:
fdb.print(20, 2)

    if not bool(lst[-1]): lst = lst[:-1]==================================================(20)      
                                                                                                                                                        (21)
    newlst = []===========================================================================(22)      
    for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes                                                      (23)
        if bool(i.strip()): newlst.append(i)==============================================(24) # users can put '' and '  ' as codes, but will be ignored.
    codes = newlst========================================================================(25)      
                                                                                                                                                        (26)
    srclines = ""=======================================================

In [ ]:
# update a new Fastdb.dbprint
fdb.dbprint(28, "turn idx of srcline into string if decode is not already a string.")
# creating a db object on whatinside with new Fastdb.dbprint
fdbw = Fastdb(whatinside, gw)
# fdbw.print(10, 1)
# we are still debuggin the same line in whatinside
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
whatinside = new
whatinside(core)
# Turn Fastdb.dbprint and whatinside back to normal
Fastdb.dbprint = fdb.orisrc
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration

                                                                                                                                                        (26)
    srclines = ""                                                                                                                                       (27)
    if type(dbcode) == int:=============================================================================================================================(28)
                                                                                           turn idx of srcline into string if decode is not already a string.
        srclines = lst[dbcode]                                                                                                                          (29)
    else:                                                                                                                                               (30)
print selected srcline with expands above----------
     

### Explore the 8th line

In [ ]:
fdb.print(20, 2)

    if not bool(lst[-1]): lst = lst[:-1]==================================================(20)      
                                                                                                                                                        (21)
    newlst = []===========================================================================(22)      
    for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes                                                      (23)
        if bool(i.strip()): newlst.append(i)==============================================(24) # users can put '' and '  ' as codes, but will be ignored.
    codes = newlst========================================================================(25)      
                                                                                                                                                        (26)
    srclines = ""=======================================================

In [ ]:
# update a new Fastdb.dbprint
fdb.dbprint(34, "Warning message if decode is not an int for idx.")
# creating a db object on whatinside with new Fastdb.dbprint
fdbw = Fastdb(whatinside, gw)
# fdbw.print(10, 1)
# we are still debuggin the same line in whatinside
new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
whatinside = new
whatinside(core)
# Turn Fastdb.dbprint and whatinside back to normal
Fastdb.dbprint = fdb.orisrc
whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration

        colmsg = colorize(" param decode has to be an int as idx.", color="y")                                                                          (32)
        print(colwarn + colmsg)                                                                                                                         (33)
#         srclines = dbcode=============================================================================================================================(34)
                                                                                                             Warning message if decode is not an int for idx.
        return                                                                                                                                          (35)
                                                                                                                                                        (36)
print selected srcline with expands above----------
     

## Make the above a function

In [ ]:
# def dbdb(pmax, # max display of src per cell
#          pt, # which part 
#          idx, # idx of srcline
#          cmt, # comment
#          *codes):  # codes
#     global whatinside, fdbw, fdb, Fastdb, gw, g
#     # print the srcode with specified length and part
#     if bool(pmax): fdb.print(pmax, pt) # if pmax == 0, then no printing src
    
#     if bool(idx):
#         # update a new Fastdb.dbprint
#         fdb.dbprint(idx, cmt, *codes)
#         # creating a db object on whatinside with new Fastdb.dbprint
#         fdbw = Fastdb(whatinside, gw)
#         # fdbw.print(10, 1)
#         # we are still debuggin the same line in whatinside
#         new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
#         whatinside = new
#         whatinside(core)
#         # Turn Fastdb.dbprint and whatinside back to normal
#         Fastdb.dbprint = fdb.orisrc
#         whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration
#     else:
#         pass

### Print the srcode at the end

In [ ]:
def dbdb(pmax, # max display of src per cell
         pt, # which part 
         idx, # idx of srcline
         cmt, # comment
         *codes, # codes
         db=False):
    global whatinside, fdbw, fdb, Fastdb, gw, g

    
    if bool(idx):
        # update a new Fastdb.dbprint
        fdb.dbprint(idx, cmt, *codes, showdbsrc=db)
        # creating a db object on whatinside with new Fastdb.dbprint
        fdbw = Fastdb(whatinside, gw)
        # fdbw.print(10, 1)
        # we are still debuggin the same line in whatinside
        new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
        whatinside = new
        whatinside(core)
        # Turn Fastdb.dbprint and whatinside back to normal
        Fastdb.dbprint = fdb.orisrc
        whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration
    else:
        pass
    
    # print the srcode with specified length and part
    if bool(pmax): fdb.print(pmax, pt) # if pmax == 0, then no printing src

### Print the srcode at the end and automatically adjust the number of srclines using idx

In [ ]:
def dbdb(idx, # idx of srcline
         cmt, # comment
         *codes, # codes
         db=False,
         full=False):
    global whatinside, fdbw, fdb, Fastdb, gw, g

    
    if bool(idx):
        # update a new Fastdb.dbprint
        fdb.dbprint(idx, cmt, *codes, showdbsrc=db)
        # creating a db object on whatinside with new Fastdb.dbprint
        fdbw = Fastdb(whatinside, gw)
        # fdbw.print(10, 1)
        # we are still debuggin the same line in whatinside
        new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
        whatinside = new
        whatinside(core)
        # Turn Fastdb.dbprint and whatinside back to normal
        Fastdb.dbprint = fdb.orisrc
        whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration
    else:
        pass
    
    totalines = len(inspect.getsource(fdb.orisrc).split('\n'))
    maxpcell = 33
    pt = idx // maxpcell
    if full:
        fdb.print()
    elif idx > maxpcell and idx % maxpcell != 0:
        fdb.print(maxpcell, pt + 1)
    else:
        fdb.print(maxpcell, 1)


### which dbprint to use db=True

In [ ]:
def dbdb(idx, # idx of srcline
         cmt, # comment
         *codes, # codes
         db=False,
         full=False):
    global whatinside, fdbw, fdb, Fastdb, gw, g

    
    if bool(idx):
        # update a new Fastdb.dbprint
#         fdb.dbprint(idx, cmt, *codes, showdbsrc=db)
        fdb.dbprint(idx, cmt, *codes, showdbsrc=db)
        print("\n")
        print('{:-<157}'.format(colorize("writing dbcodes into Fastdb.dbprint is done above", color="y")))
        print("\n")
        # creating a db object on whatinside with new Fastdb.dbprint
        fdbw = Fastdb(whatinside, gw)
        # fdbw.print(10, 1)
        # we are still debuggin the same line in whatinside
        new = fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)", showdbsrc=db)
        print("\n")
        print('{:-<157}'.format(colorize("using db Fastdb.dbprint to create db whatinside is done", color="y")))
        print("\n")
        whatinside = new
        whatinside(core)
        print("\n")
        print('{:-<157}'.format(colorize("using db whatinside to run example on core above", color="y")))
        print("\n")        
        # Turn Fastdb.dbprint and whatinside back to normal
        Fastdb.dbprint = fdb.orisrc
        whatinside = fdbw.orisrc # Important! remember to bring whatinside back to normal after each srcline exploration
    else:
        pass
    
    totalines = len(inspect.getsource(fdb.orisrc).split('\n'))
    maxpcell = 33
    pt = idx // maxpcell
    if full:
        fdb.print()
    elif idx > maxpcell and idx % maxpcell != 0:
        fdb.print(maxpcell, pt + 1)
    else:
        fdb.print(maxpcell, 1)


In [ ]:
dbdb(36, "loop every srcline with idx", "len(lst)")

#         srclines = dbcode                                                                                                                             (34)
        return                                                                                                                                          (35)
    for idx, l in zip(range(len(lst)), lst):                                                                                                            (37)
                                                                                                                                                        (38)
print selected srcline with expands above----------


writing dbcodes into Fastdb.dbprint is done above---------------------------------------------------------------------------------------------------


             ):                                                                                                                                         (7)
    'Check 

In [ ]:
dbdb(38, "When a srcline is under investigation")

                                                                                                                                                        (36)
    for idx, l in zip(range(len(lst)), lst):                                                                                                            (37)
        if bool(l.strip()) and l.strip() in srclines and idx == dbcode:                                                                                 (39)
                                                                                                                                                        (40)
print selected srcline with expands above----------


writing dbcodes into Fastdb.dbprint is done above---------------------------------------------------------------------------------------------------


             ):                                                                                                                                         (7)
    'Check 

In [ ]:
dbdb(40, "When there is codes to run", "codes")

                                                                                                                                                        (38)
        if bool(l.strip()) and l.strip() in srclines and idx == dbcode:                                                                                 (39)
            if len(codes) > 0:                                                                                                                          (41)
                numindent = len(l) - len(l.lstrip()) # make sure indent not messed up by trailing spaces                                                (42)
print selected srcline with expands above----------


writing dbcodes into Fastdb.dbprint is done above---------------------------------------------------------------------------------------------------


             ):                                                                                                                                         (7)
    'Check 

In [ ]:
dbdb(53, "what does dbprintinsert line look like", "dbsrc", "dbsrc + l")

                                                                                                                                                        (51)
                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'                                                                                   (52)
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'==========================================================================================(53)
                                                                                                                       what does dbprintinsert line look like
                dbsrc = dbsrc + l + '\n'                                                                                                                (54)
            else:                                                                                                                                       (55)
print selected srcline with expands above----------


wri

In [ ]:
dbdb(57, "what to print at the end")

            else:                                                                                                                                       (55)
                dbsrc = dbsrc + l + '\n'                                                                                                                (56)
        elif bool(l.strip()) and idx + 1 == len(lst):                                                                                                   (58)
            dbsrc = dbsrc + l                                                                                                                           (59)
print selected srcline with expands above----------


writing dbcodes into Fastdb.dbprint is done above---------------------------------------------------------------------------------------------------


             ):                                                                                                                                         (7)
    'Check 

In [ ]:
dbdb(63, "print out the srcode with dbprintinsert inserted")

        elif bool(l.strip()): # make sure pure indentation + \n is ignored                                                                              (61)
            dbsrc = dbsrc + l + '\n'                                                                                                                    (62)
    if showdbsrc: # added to debug                                                                                                                      (64)
        print('{:-<60}'.format(colorize("showdbsrc=Start", color="y")))                                                                                 (65)
print selected srcline with expands above----------


writing dbcodes into Fastdb.dbprint is done above---------------------------------------------------------------------------------------------------


             ):                                                                                                                                         (7)
    'Check 

In [ ]:
dbdb(75, "check status of locals, Fastdb.dbprint and fdb.dbprint", db=True)

                print(l + "="*(totallen-lenl-lenidx) + "(db)")                                                                                          (73)
            else:                                                                                                                                       (74)
                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")============================================================================(75)
                                                                                                       check status of locals, Fastdb.dbprint and fdb.dbprint
        print(f"before exec, locals(): {list(locals().keys())}")                                                                                        (76)
                                                                                                                                                        (77)
print selected srcline with expands above----------
showd

In [ ]:
fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)", showdbsrc=True)

             ):                                                                                                                                         (7)
    'Check what inside a module: `__all__`, functions, classes, builtins, and callables'                                                                (8)
    dun_all = len(mo.__all__) if hasattr(mo, "__all__") else 0==========================================================================================(9)
                                                                                                                                count num of funcs in __all__
    funcs = inspect.getmembers(mo, inspect.isfunction)                                                                                                  (10)
    classes = inspect.getmembers(mo, inspect.isclass)                                                                                                   (11)
print selected srcline with expands above----------
showdbsr

<function fastdebug.utils.whatinside(mo, dun: bool = False, func: bool = False, clas: bool = False, bltin: bool = False, lib: bool = False, cal: bool = False)>

In [ ]:
dbdb(79, "run exec to create the dbsrc and store in locals()", "list(locals().keys())", "list(env.keys())", "list(self.outenv.keys())")

                                                                                                                                                        (77)
    exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above                                                                  (78)
    ====================================================================================================================================================(79)
                                                                                                           run exec to create the dbsrc and store in locals()
    if showdbsrc:                                                                                                                                       (80)
        print(f"after exec, locals(): {list(locals().keys())}")                                                                                         (81)
print selected srcline with expands above----------


wri

In [ ]:
dbdb(83, "return the dbsrc using locals()", "list(locals().keys())", "list(env.keys())", "list(self.outenv.keys())", "self.orisrc.__name__", db=True, full=True)

        print(f"after exec, locals(): {list(locals().keys())}")                                                                                         (81)
        print(f'self.orisrc.__name__: {self.orisrc.__name__}')                                                                                          (82)
        print(f'locals()[self.orisrc.__name__]: {locals()[self.orisrc.__name__]}')======================================================================(83)
                                                                                                                              return the dbsrc using locals()
        print('{:-<60}'.format(colorize("showdbsrc=End", color="y")))                                                                                   (84)
                                                                                                                                                        (85)
print selected srcline with expands above----------
showd

### How to check whether two vars refer to the same object

In [ ]:
fdb.dbprint is Fastdb.dbprint

In [ ]:
fdb.dbprint is fdb.orisrc

In [ ]:
Fastdb.dbprint is fdb.orisrc

## Make the above into a class and methods

In [ ]:
from fastdebug.core import *
from fastdebug.utils import whatinside
from fastcore.basics import *

In [ ]:
Fastdb.dbprint.__qualname__

'Fastdb.dbprint'

In [ ]:
class DBDB():
    "Using Fastdb to debug Fastdb"
    def __init__(self, 
                 clsfunc=Fastdb.dbprint, # name of src code you are exploring
                 egfunc=whatinside): # env variables needed for exploring the source code, e.g., g = globals()
        self.oriclsfunc = clsfunc
        self.oriegfunc = egfunc
        self.cls = Fastdb # to use exec later
        self.clsfunc = clsfunc
        self.egfunc = egfunc
        
        self.g_clsfunc = {}
        self.g_clsfunc.update(clsfunc.__globals__)
        self.g_egfunc = {}
        self.g_egfunc.update(egfunc.__globals__)
        
        self.fdb = self.cls(clsfunc, self.g_clsfunc)

#         self.margin = 157
#         self.outenv = env
#         self.cmts = {}

In [ ]:
@patch
def print(self:DBDB,
          pmax=10, # max display of src per cell
          pt=1): # which part
    
    self.fdb.print(pmax, pt) # if pmax == 0, then no printing src

In [ ]:
@patch
def dbdb(self:DBDB,
         idx, # idx of srcline
         cmt, # comment
         *codes): # expressions to evaluate

    # Turn Fastdb.dbprint and whatinside back to normal
    self.clsfunc = self.oriclsfunc
    self.egfunc = self.oriegfunc # Important! remember to bring whatinside back to normal after each srcline exploration

    # update a new Fastdb.dbprint
    self.fdb.dbprint(idx, cmt, *codes)
    # creating a db object on whatinside with new Fastdb.dbprint
    self.fdbw = self.cls(self.egfunc, self.g_egfunc)
    # fdbw.print(10, 1)
    # we are still debuggin the same line in whatinside
    new = self.fdbw.dbprint(9, "count num of funcs in __all__", "type(mo)")
    self.egfunc = new
    self.egfunc(core)
    

In [ ]:
obj = DBDB()

In [ ]:
obj.print()

@patch====================================================================================(0)       
def dbprint(self:Fastdb, =================================================================(1)       
            dbcode:int, # idx of a srcline under investigation, can only be int===========(2)       
            cmt:str, # comment added to the srcline=======================================(3)       
            *codes, # a list of expressions (str) you write to be evaluated above the srcline                                                           (4)
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated                                                            (5)
            showdbsrc:bool=False): # display dbsrc========================================(6)       
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating"                       (7)
                           

In [ ]:
obj.dbdb(36, "loop every srcline with idx", "len(lst)")

        return                                                                                                                                          (34)
                                                                                                                                                        (35)
    for idx, l in zip(range(len(lst)), lst):============================================================================================================(36)
                                                                                                                                  loop every srcline with idx
                                                                                                                                                        (37)
        if bool(l.strip()) and l.strip() in srclines and idx == dbcode:                                                                                 (38)
             ):                                          

In [ ]:
obj.dbdb(38, "when srcline match decode idx")

    for idx, l in zip(range(len(lst)), lst):                                                                                                            (36)
                                                                                                                                                        (37)
        if bool(l.strip()) and l.strip() in srclines and idx == dbcode:=================================================================================(38)
                                                                                                                                when srcline match decode idx
                                                                                                                                                        (39)
            if len(codes) > 0:                                                                                                                          (40)


                                                       

In [ ]:
obj.print(30,2)

        colwarn = colorize("Warning!", color="r")=========================================(30)      
        colmsg = colorize(" param decode has to be an int as idx.", color="y")============(31)      
        print(colwarn + colmsg)===========================================================(32)      
#         srclines = dbcode===============================================================(33)      
        return============================================================================(34)      
                                                                                                                                                        (35)
    for idx, l in zip(range(len(lst)), lst):==============================================(36) # loop every srcline with idx
                                                                                                                                                        (37)
        if bool(l.strip()) and l.strip() in srclines and

In [ ]:
obj.dbdb(40, "when there is code to be evaluated", "codes", "len(codes)")

        if bool(l.strip()) and l.strip() in srclines and idx == dbcode:                                                                                 (38)
                                                                                                                                                        (39)
            if len(codes) > 0: =========================================================================================================================(40)
                                                                                                                           when there is code to be evaluated
                numindent = len(l) - len(l.lstrip()) # make sure indent not messed up by trailing spaces                                                (41)
                dbcodes = "dbprintinsert("                                                                                                              (42)
             ):                                          

In [ ]:
obj.print(20,3)

            if len(codes) > 0: ===========================================================(40) # when there is code to be evaluated
                numindent = len(l) - len(l.lstrip()) # make sure indent not messed up by trailing spaces                                                (41)
                dbcodes = "dbprintinsert("================================================(42)      
                count = 1=================================================================(43)      
                for c in codes:===========================================================(44)      
                    if count == len(codes):===============================================(45)      
                        dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"===========(46)      
                    else:=================================================================(47)      
                        dbcodes = dbcodes + '"' + c + '"' + ","===========================(48)      
    

In [ ]:
obj.clsfunc == Fastdb.dbprint

False

In [ ]:
obj.clsfunc

<function fastdebug.core.Fastdb.dbprint(self: fastdebug.core.Fastdb, dbcode: int, cmt: str, *codes, expand: int = 2, showdbsrc: bool = False)>

In [ ]:
obj.dbdb(53, "add codes as strings into dbprintinser", "dbsrc", "dbsrc + l")

                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'                                                                                   (51)
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'                                                                                          (52)
                dbsrc = dbsrc + l + '\n'     ===========================================================================================================(53)
                                                                                                                       add codes as strings into dbprintinser
            else:                                                                                                                                       (54)
                dbsrc = dbsrc + l + '\n'                                                                                                                (55)


                                                       